In [19]:
!pip install langchain-groq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    groq_api_key = "use your own generateed API"
)

In [21]:
response = llm.invoke("what the capital of india")

In [22]:
print(response.content)

The capital of India is New Delhi. It is one of the 28 states and 8 Union territories of India. New Delhi serves as the seat of government and the center of politics in India. It is known for its rich history, diverse culture, and numerous tourist attractions such as the Red Fort, Jama Masjid, and Humayun's Tomb. New Delhi is also home to many important institutions of the Indian government, including the Parliament of India and the Rashtrapati Bhavan, the official residence of the President of India.


## Scrapping data using WebBaseLoader class of LLMs- it use beautifulsoup behind the scene.

In [23]:
from langchain_community.document_loaders import WebBaseLoader
site = "https://www.metacareers.com/v2/jobs/1008290497511075/"
loader = WebBaseLoader(site)
page_data = loader.load().pop().page_content
print(page_data)



Software Engineer, Language - Generative AI | Meta Careers






















Skip to main contentJobsAreas of WorkLocations Career Programs  How We Work BlogJobsBackAreas of WorkBy TechnologyFacebookInstagramMessengerBy TechnologyMeta QuestWhatsAppWorkplaceBy FieldAdvertising TechnologyAR and VRArtificial IntelligenceBusiness Development and PartnershipsCommunications and Public PolicyCreativeData and AnalyticsDesign and User ExperienceEnterprise EngineeringGlobal OperationsBy FieldInfrastructureLegal, Finance, Facilities and AdministrationPeople and RecruitingProduct ManagementResearchSales and MarketingSecuritySoftware EngineeringTechnical Program ManagementLocationsBackCareer ProgramsStudents & GradsPathway ProgramsBackHow We WorkCultureDiversityVeteransLeadershipInnovationBenefitsBlogCreate Career ProfileYou can create a Career Profile to get job suggestions, prepare for the interview process, and more.Create Career ProfileAlready have a Career Profile?Log inStudents & GradsP

## write prompt temp to convert this extracted texrt into desire json formate for further use.

In [24]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `company name`,`role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [31]:
print(res.content)

[
  {
    "company name": "Meta",
    "role": "Software Engineer, Language - Generative AI",
    "experience": "5+ years of industry, academic or government laboratory experience",
    "skills": [
      "Python",
      "PyTorch",
      "Machine Learning",
      "Generative AI",
      "LLM"
    ],
    "description": "Meta is seeking a Software Engineer to join our Meta AI foundation team. Responsibilities include designing methods, tools, and infrastructure to push forward the state of the art in large language models and their applications, applying AI technologies to develop and optimize Meta’s products and services, contributing to experiments, adapting standard machine learning methods to best exploit modern parallel environments, and working with a large and globally distributed team. The candidate should have experience with developing machine learning models at scale from inception to business impact, and industrial experience in generative AI and LLM."
  }
]


## convert into json formate

In [32]:
import json

# Data extracted
data = [
  {
    "company name": "Meta",
    "role": "Software Engineer",
    "experience": "5+ years of industry, academic or government laboratory experience",
    "skills": [
      "Python",
      "PyTorch",
      "Generative AI",
      "LLM (Large Language Models)",
      "NLP/NLU (Natural Language Processing/Understanding)"
    ],
    "description": "Meta is seeking a Software Engineer to join our Meta AI foundation team. Responsibilities include designing methods, tools, and infrastructure to push forward the state of the art in large language models and their applications, applying AI technologies to develop and optimize Meta’s products and services, contributing to experiments, adapting standard machine learning methods to best exploit modern parallel environments, and working with a large and globally distributed team. The candidate should have experience with developing machine learning models at scale from inception to business impact and industrial experience in generative AI and LLM."
  }
]

# Convert the data to a JSON file
json_file = "job_posting.json"
with open(json_file, "w") as file:
    json.dump(data, file, indent=4)

print(f"JSON file '{json_file}' created successfully.")


JSON file 'job_posting.json' created successfully.


In [33]:
import pandas as pd

# Load the JSON file into a DataFrame
job = pd.read_json('job_posting.json')

In [34]:
job

,company name,role,experience,skills,description
0,Meta,Software Engineer,"5+ years of industry, academic or government l...","[Python, PyTorch, Generative AI, LLM (Large La...",Meta is seeking a Software Engineer to join ou...


In [36]:
# get only skills section

skills = job.iloc[0,3]
skills

['Python',
 'PyTorch',
 'Generative AI',
 'LLM (Large Language Models)',
 'NLP/NLU (Natural Language Processing/Understanding)']

In [38]:
company_name = job.iloc[0,0]
company_name

'Meta'

In [45]:
job_tital = job.iloc[0,1]
job_tital

'Software Engineer'

In [39]:
# Convert the list of skills to a single string
skills_str = ', '.join(skills)

# Print the result
print(skills_str)

Python, PyTorch, Generative AI, LLM (Large Language Models), NLP/NLU (Natural Language Processing/Understanding)


In [40]:
import pandas as pd

# df = pd.read_csv("./project-genai-cold-email-generator-main/my_portfolio.csv")
df = pd.read_csv("github_repositories.csv")
df

,Repositories,Link
0,Generative_AI,https://github.com/v26199/Generative_AI
1,ML_Projects,https://github.com/v26199/ML_Projects
2,E-Commerce-Product-Recommodation,https://github.com/v26199/E-Commerce-Product-R...
3,Credit-card-financial-dashboard-using-Power-BI,https://github.com/v26199/Credit-card-financia...
4,churn_prediction_lazyclassifier_93-accuracy,https://github.com/v26199/churn_prediction_laz...
5,RAG-using-Ollama-on-Gemma-7B-LLMs,https://github.com/v26199/RAG-using-Ollama-on-...
6,LLMs_finetunning,https://github.com/v26199/LLMs_finetunning
7,NLP_Projects,https://github.com/v26199/NLP_Projects
8,facemask-detection,https://github.com/v26199/facemask-detection
9,heart-disease-prediction,https://github.com/v26199/heart-disease-predic...


In [41]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore2')
collection = client.get_or_create_collection(name="portfolio2")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Repositories"],
                       metadatas={"links": row["Link"]},
                       ids=[str(uuid.uuid4())])

In [42]:
links = collection.query(query_texts=skills_str, n_results=5).get('metadatas', [])
links

[[{'links': 'https://github.com/v26199/NLP_Projects'},
  {'links': 'https://github.com/v26199/Generative_AI'},
  {'links': 'https://github.com/v26199/Data-Science-Resources'},
  {'links': 'https://github.com/v26199/ML_Projects'},
  {'links': 'https://github.com/v26199/Mastering_Jupyter'}]]

In [46]:
skills

['Python',
 'PyTorch',
 'Generative AI',
 'LLM (Large Language Models)',
 'NLP/NLU (Natural Language Processing/Understanding)']

In [47]:
# prompt_email = PromptTemplate.from_template(
#         """
#         ### JOB DESCRIPTION:
#         {job_description}
        
#         ### INSTRUCTION:
#         You are Vishal, a business development executive at Maxgen Technologies. Maxgen is an AI & Software Consulting company dedicated to facilitating
#         the seamless integration of business processes through automated tools. 
#         Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
#         process optimization, cost reduction, and heightened overall efficiency. 
#         Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
#         in fulfilling their needs.
#         Also add the most relevant ones from the following links to showcase Maxgen's portfolio: {link_list}
#         Remember you are Vishal, BDE at Maxgen . 
#         Do not provide a preamble.
#         ### EMAIL (NO PREAMBLE):
        
#         """
#         )

prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        Dear [Hiring Manager’s Name],

        My name is Vishal Patel, and I am an experienced Data Scientist with a strong focus on machine learning, NLP, and generative LLM models. I am writing to express my interest in the {Job_title} position at {Company_Name}, which I discovered through your company's career job posting.

        With over 5 years of experience in data science and machine learning, I believe my background makes me an excellent fit for this role.

        My recent project involved developing an advanced LLM-based cold email generator that automated job listing extraction and personalized email generation. This innovation resulted in a 30% increase in response rates and a 25% rise in project engagement, demonstrating my ability to drive significant improvements in outreach and efficiency.

        My expertise aligns perfectly with the skills required for this role. I specialize in Python, machine learning, and NLP, with a proven track record of optimizing AI solutions and deploying them effectively. I am adept at leveraging modern technologies to address complex challenges and enhance business processes.

        For your convenience, I have included the most relevant portfolio examples from the following links to showcase my capabilities:
        * {link_list}

        I would welcome the opportunity to discuss how my skills and experience align with your needs. I look forward to the possibility of contributing to your team.

        Best regards,

        Vishal Patel
        Data Scientist

        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        """
        )




chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job),"Job_title" :job_tital ,"Company_Name": company_name, "link_list": links})
print(res.content)

Subject: Application for Software Engineer Position at Meta

Dear [Hiring Manager’s Name],

My name is Vishal Patel, and I am an experienced Data Scientist with a strong focus on machine learning, NLP, and generative LLM models. I am writing to express my interest in the Software Engineer position at Meta, which I discovered through your company's career job posting.

With over 5 years of experience in data science and machine learning, I believe my background makes me an excellent fit for this role. My recent project involved developing an advanced LLM-based cold email generator that automated job listing extraction and personalized email generation. This innovation resulted in a 30% increase in response rates and a 25% rise in project engagement, demonstrating my ability to drive significant improvements in outreach and efficiency.

My expertise aligns perfectly with the skills required for this role. I specialize in Python, machine learning, and NLP, with a proven track record of op